[View in Colaboratory](https://colab.research.google.com/github/taiki323/kaggle_training/blob/master/Google_Analytics_Customer_Revenue_Prediction.ipynb)

# 課題
- Google Merchandise Storeの顧客データを分析して、顧客あたりの収益を予測する。

- 各fullVisitorIdに対して、PredictedLogRevenueで総収入の自然対数を予測する。RMSEで評価

![target](https://github.com/taiki323/image_house/blob/master/target1.PNG?raw=true)

# セットアップ

In [0]:
!pip install kaggle　
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
# !mkdir .kaggle
!mkdir ~/.kaggle
%cd /content
!mkdir work
%cd work
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle competitions download -c ga-customer-revenue-prediction

mkdir: cannot create directory ‘/root/.kaggle’: File exists
/content
mkdir: cannot create directory ‘work’: File exists
/content/work
User cancelled operation
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 11, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/kaggle/cli.py", line 59, in main
    out = None
KeyboardInterrupt


In [0]:
!unzip '*.zip'

Archive:  sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N

Archive:  train.csv.zip

Archive:  test.csv.zip

3 archives were successfully processed.


In [0]:
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold

# 前処理

## データ確認
１回のストア訪問につき、１行のデータになる。

train:90万データ   
test: 80万データ
- fullVisitorId: 一意なユーザID
- channelGrouping: ユーザがストアにアクセスした経路(アフィリエイトなど)
- date: ストアに訪れた日付
- device: ユーザが使用したデバイス
- geoNetwork: ユーザがいる地域
- sessionId: セッションID
- socialEngagementType: Not Socially Engagedしかデータない
- totals: セッション全体の集計値
- trafficSource: トラフィックソースの情報
- visitId: セッションID。ユーザにのみ一意。
- visitNumber: セッション番号。初回アクセスなら1。
- visitStartTime: ストアに訪れた時間

In [0]:
#データ読み込み
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(csv_path, dtype={'fullVisitorId': 'str'}, nrows=nrows)

    for column in JSON_COLUMNS:
        df = df.join(pd.DataFrame(df.pop(column).apply(pd.io.json.loads).values.tolist(), index=df.index))

    return df
  
train = load_df("/content/work/train.csv")
test = load_df("/content/work/test.csv")

In [0]:
print(train.shape)
print(test.shape)
train.head()

(903653, 50)
(804684, 48)


,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserSize,...,visits,adContent,adwordsClickInfo,campaign,campaignCode,isTrueDirect,keyword,medium,referralPath,source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,1,NaN,{'criteriaParameters': 'not available in demo ...,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,1,NaN,{'criteriaParameters': 'not available in demo ...,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,1,NaN,{'criteriaParameters': 'not available in demo ...,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,1,NaN,{'criteriaParameters': 'not available in demo ...,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,1,NaN,{'criteriaParameters': 'not available in demo ...,(not set),NaN,True,(not provided),organic,NaN,google


In [0]:
#trainデータにあって、testデータに無いカラム
diff_col = list(set(train.columns) - set(test.columns))
print(diff_col)

['transactionRevenue', 'campaignCode']


## データクレイジング

### 不要なカラムの除去

In [0]:
#全て同じ値のカラムを削除
train['adwordsClickInfo'] = train['adwordsClickInfo'].astype(str)
test['adwordsClickInfo'] = test['adwordsClickInfo'].astype(str)
train = train.drop('campaignCode',axis=1)

for column in train.columns:
  if len(train[column].value_counts()) == 1:
    train = train.drop(column,axis=1)
    test = test.drop(column,axis=1)
    print(column)

socialEngagementType
browserSize
browserVersion
flashVersion
language
mobileDeviceBranding
mobileDeviceInfo
mobileDeviceMarketingName
mobileDeviceModel
mobileInputSelector
operatingSystemVersion
screenColors
screenResolution
cityId
latitude
longitude
networkLocation
bounces
newVisits
visits
isTrueDirect


In [0]:
#欠損値を確認
def kesson_table (df):
  null_val = df.isnull().sum()
  percent = (null_val / len(df)) * 100
  kesson_table = pd.concat([null_val, percent],axis=1)
  kesson_table_ren_columns = kesson_table.rename(columns={0:'欠損値', 1:'%'})
  return kesson_table_ren_columns

print(kesson_table(train))
print(kesson_table(test))

                       欠損値          %
channelGrouping          0   0.000000
date                     0   0.000000
fullVisitorId            0   0.000000
sessionId                0   0.000000
visitId                  0   0.000000
visitNumber              0   0.000000
visitStartTime           0   0.000000
browser                  0   0.000000
deviceCategory           0   0.000000
isMobile                 0   0.000000
operatingSystem          0   0.000000
city                     0   0.000000
continent                0   0.000000
country                  0   0.000000
metro                    0   0.000000
networkDomain            0   0.000000
region                   0   0.000000
subContinent             0   0.000000
hits                     0   0.000000
pageviews              100   0.011066
transactionRevenue  892138  98.725728
adContent           892707  98.788694
adwordsClickInfo         0   0.000000
campaign                 0   0.000000
keyword             502929  55.655102
medium      

In [0]:
#半分以上データが欠損しているカラムを削除
#transactionRevenueは目的変数なので削除しない
del_columns = ["referralPath","keyword", "adContent"]
train = train.drop(del_columns, axis=1)
test = test.drop(del_columns, axis=1)

In [0]:
#予測に関係のないデータを削除
unwanted = ["browser", "adwordsClickInfo"]
train = train.drop(unwanted, axis=1)
test = test.drop(unwanted, axis=1)

In [0]:
#transactionRevenueのNaNを0埋め
train['transactionRevenue'] = train['transactionRevenue'].fillna(0)

In [0]:
print(train.shape)
print(test.shape)

(903653, 23)
(804684, 22)


## プロット

### transactionRevenueの散布図

In [0]:
train['transactionRevenue'] = train['transactionRevenue'].astype(float)
gdf = train.groupby('fullVisitorId')['transactionRevenue'].sum().reset_index()

In [0]:
gdf.sort_values('transactionRevenue')

,fullVisitorId,transactionRevenue
0,0000010278554503158,0.000000e+00
474471,6648333936862347204,0.000000e+00
474472,664833859746538014,0.000000e+00
474473,6648399661474388281,0.000000e+00
474474,6648402145907760889,0.000000e+00
474475,6648428571934524032,0.000000e+00
474476,6648438211615859760,0.000000e+00
474477,6648448038004646979,0.000000e+00
474478,6648451811895664466,0.000000e+00
474479,6648507502027055198,0.000000e+00
